# Convolutional Neural Network

### Importing the libraries

In [56]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [57]:
tf.__version__

'2.17.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [58]:
target_size = (60, 60) # all images will be resized to 150x150 (try 64x64)

train_datagen = ImageDataGenerator(
    rescale=1./255, # Set all pixels values between 0 and 1
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

training_set = train_datagen.flow_from_directory(
        'dataset/training_set',  # this is the target directory
        target_size=target_size,
        batch_size=32,
        class_mode='binary') # use binary since we have only two options, cat or dog

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [59]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=target_size,
        batch_size=32,
        class_mode='binary',
        shuffle=False)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [60]:
cnn = tf.keras.Sequential()

### Step 1 - Convolution

In [61]:
cnn.add(
    tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=3,
        activation='relu',
        input_shape=(*target_size, 3)))

### Step 2 - Pooling

In [62]:
cnn.add(
    tf.keras.layers.MaxPool2D(
        strides=2,
        pool_size=(2, 2)))

### Adding a second convolutional layer

In [63]:
# The same as prev, but without input_shape since it should be set only in first layer
cnn.add(
    tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=3,
        activation='relu'))
cnn.add(
    tf.keras.layers.MaxPool2D(
        strides=2,
        pool_size=(2, 2)))

### Step 3 - Flattening

In [64]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [65]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [66]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [67]:
# cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
            loss='binary_crossentropy', 
            metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [68]:
cnn.fit(training_set, validation_data=test_set, epochs=25)

Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 51ms/step - accuracy: 0.5285 - loss: 0.6919 - val_accuracy: 0.6215 - val_loss: 0.6496
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 51ms/step - accuracy: 0.6417 - loss: 0.6439 - val_accuracy: 0.5880 - val_loss: 0.6652
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 51ms/step - accuracy: 0.6736 - loss: 0.6037 - val_accuracy: 0.6975 - val_loss: 0.5755
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 50ms/step - accuracy: 0.6881 - loss: 0.5887 - val_accuracy: 0.6700 - val_loss: 0.6070
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - accuracy: 0.7121 - loss: 0.5624 - val_accuracy: 0.7125 - val_loss: 0.5640
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 47ms/step - accuracy: 0.7129 - loss: 0.5591 - val_accuracy: 0.7325 - val_loss: 0.5349
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - accuracy: 0.7125 - loss: 0.5488 - val_accuracy: 0.7415 - val_loss: 0.5263
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - accuracy: 0.7310 - loss: 0.5420 - 

## Part 4 - Making a single prediction

In [69]:
import numpy as np

test_image = tf.keras.utils.load_img('dataset/single_prediction/cat_or_dog_5.jpg', target_size=target_size)
test_image = tf.keras.utils.img_to_array(test_image)
# because of the batch_size we need to create a dump ones to fill it batch up to 32
test_image = np.expand_dims(test_image, axis=0)

result = cnn.predict(test_image)

# result is a batch, so result[0] is the first batch 
# and result[0][0] is the first element of the first batch
probability = result[0][0]

print(training_set.class_indices)


prediction = 'cat' if result[0][0] == 0 else 'dog'
print( probability)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
{'cats': 0, 'dogs': 1}
0.0
